# CNN Training

In [1]:
'''Trains a simple convnet on the MNIST dataset and embeds test data.
The test data is embedded using the weights of the final dense layer, just
before the classification head. This embedding can then be visualized using
TensorBoard's Embedding Projector.
'''

from __future__ import print_function

from os import makedirs
from os.path import exists, join

import keras
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import numpy as np


batch_size = 128
num_classes = 10
epochs = 10
log_dir = './logs-cnn'

if not exists(log_dir):
    makedirs(log_dir)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
# save class labels to disk to color data points in TensorBoard accordingly
with open(join(log_dir, 'metadata.tsv'), 'w') as f:
    np.savetxt(f, y_test)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [3]:
tensorboard = TensorBoard(batch_size=batch_size,
                          embeddings_freq=1,
                          embeddings_layer_names=['features'],
                          embeddings_metadata='metadata.tsv',
                          embeddings_data=x_test,
                          histogram_freq=0, 
                          write_graph=True, 
                          write_grads=False, 
                          write_images=False, 
                          update_freq='epoch',
                          log_dir=log_dir )

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu', name='features'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [4]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          callbacks=[tensorboard ],
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# You can now launch tensorboard with `tensorboard --logdir=./logs` on your
# command line and then go to http://localhost:6006/#projector to view the
# embeddings

W0703 11:13:00.422507  4116 deprecation_wrapper.py:119] From C:\Users\Noor\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0703 11:13:00.710800  4116 deprecation_wrapper.py:119] From C:\Users\Noor\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0703 11:13:00.828517  4116 deprecation_wrapper.py:119] From C:\Users\Noor\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0703 11:13:01.098136  4116 deprecation_wrapper.py:119] From C:\Users\Noor\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0703 11:13:01.109963  4116 deprecation_wrapper.py:119] From C:\Users\Noor\Anaconda

Train on 60000 samples, validate on 10000 samples


W0703 11:13:03.699476  4116 deprecation_wrapper.py:119] From C:\Users\Noor\Anaconda3\lib\site-packages\keras\callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0703 11:13:03.699476  4116 deprecation_wrapper.py:119] From C:\Users\Noor\Anaconda3\lib\site-packages\keras\callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0703 11:13:04.016844  4116 deprecation_wrapper.py:119] From C:\Users\Noor\Anaconda3\lib\site-packages\keras\callbacks.py:887: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.



Epoch 1/10
60000/60000 [==============================] - 176s 3ms/step - loss: 0.2780 - acc: 0.9139 - val_loss: 0.0593 - val_acc: 0.9802
Epoch 2/10
60000/60000 [==============================] - 175s 3ms/step - loss: 0.0913 - acc: 0.9731 - val_loss: 0.0394 - val_acc: 0.9860
Epoch 3/10
60000/60000 [==============================] - 173s 3ms/step - loss: 0.0680 - acc: 0.9804 - val_loss: 0.0369 - val_acc: 0.9878
Epoch 4/10
60000/60000 [==============================] - 172s 3ms/step - loss: 0.0552 - acc: 0.9837 - val_loss: 0.0323 - val_acc: 0.9882
Epoch 5/10
60000/60000 [==============================] - 174s 3ms/step - loss: 0.0476 - acc: 0.9859 - val_loss: 0.0284 - val_acc: 0.9910
Epoch 6/10
60000/60000 [==============================] - 173s 3ms/step - loss: 0.0423 - acc: 0.9871 - val_loss: 0.0275 - val_acc: 0.9904


W0703 11:33:42.772357  4116 deprecation.py:323] From C:\Users\Noor\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


Epoch 7/10
60000/60000 [==============================] - 174s 3ms/step - loss: 0.0374 - acc: 0.9885 - val_loss: 0.0269 - val_acc: 0.9904
Epoch 8/10
60000/60000 [==============================] - 198s 3ms/step - loss: 0.0361 - acc: 0.9889 - val_loss: 0.0273 - val_acc: 0.9911
Epoch 9/10
60000/60000 [==============================] - 196s 3ms/step - loss: 0.0312 - acc: 0.9910 - val_loss: 0.0328 - val_acc: 0.9892
Epoch 10/10
60000/60000 [==============================] - 201s 3ms/step - loss: 0.0302 - acc: 0.9907 - val_loss: 0.0262 - val_acc: 0.9909
Test loss: 0.026161733050311522
Test accuracy: 0.9909


# Deep NN Training

In [5]:
'''Trains a simple convnet on the MNIST dataset and embeds test data.
The test data is embedded using the weights of the final dense layer, just
before the classification head. This embedding can then be visualized using
TensorBoard's Embedding Projector.
'''

from __future__ import print_function

from os import makedirs
from os.path import exists, join

import keras
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import numpy as np

# TensorFlow and tf.keras
import tensorflow as tf
# To ignore keep_dims warning
tf.logging.set_verbosity(tf.logging.ERROR)

batch_size = 128
num_classes = 10
epochs = 10
log_dir = './logs-DNN'

if not exists(log_dir):
    makedirs(log_dir)

# the data, split between train and test sets
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#train_images2 = train_images.reshape(60000, 784)     
#test_images2 = test_images.reshape(10000, 784)

#train_images2 = train_images2.astype('float32')     
#test_images2 = test_images2.astype('float32')     

train_images2 = train_images / 255.0
test_images2 = test_images / 255.0

# convert class vectors to binary class matrices
train_labels2 = keras.utils.to_categorical(train_labels, num_classes)
test_labels2 = keras.utils.to_categorical(test_labels, num_classes)

In [6]:
# save class labels to disk to color data points in TensorBoard accordingly
with open(join(log_dir, 'metadata2.tsv'), 'w') as f:
    np.savetxt(f, test_labels2)

In [7]:
tensorboard2 = TensorBoard(batch_size=batch_size,
                          embeddings_freq=1,
                          embeddings_layer_names=['features'],
                          embeddings_metadata='metadata.tsv',
                          embeddings_data=test_images2,
                          histogram_freq=0, 
                          write_graph=True, 
                          write_grads=False, 
                          write_images=False, 
                          update_freq='epoch',
                          log_dir=log_dir )

In [8]:
model2 = Sequential()
model2.add(Flatten(input_shape=(28, 28))),
model2.add(Dense(128, activation=tf.nn.relu, name='features'))
# output = relu (dot (W, input) + bias)
model2.add(Dense(10, activation=tf.nn.softmax))

In [9]:
model2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [10]:
model2.fit(train_images2, train_labels2,
          batch_size=batch_size,
          callbacks=[tensorboard2],
          epochs=epochs,
          verbose=1,
          validation_data=(test_images2, test_labels2)
          )

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.3579 - acc: 0.9008 - val_loss: 0.2163 - val_acc: 0.9362
Epoch 2/10
60000/60000 [==============================] - 3s 45us/step - loss: 0.1721 - acc: 0.9512 - val_loss: 0.1382 - val_acc: 0.9591
Epoch 3/10
60000/60000 [==============================] - 3s 49us/step - loss: 0.1270 - acc: 0.9638 - val_loss: 0.1128 - val_acc: 0.9665
Epoch 4/10
60000/60000 [==============================] - ETA: 0s - loss: 0.1015 - acc: 0.971 - 3s 49us/step - loss: 0.1012 - acc: 0.9714 - val_loss: 0.1018 - val_acc: 0.9702
Epoch 5/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.0844 - acc: 0.9762 - val_loss: 0.0930 - val_acc: 0.9716
Epoch 6/10
60000/60000 [==============================] - 3s 46us/step - loss: 0.0723 - acc: 0.9797 - val_loss: 0.0865 - val_acc: 0.9736
Epoch 7/10
60000/60000 [==============================] - 3s 46us/step - loss: 0.0626 - ac

In [11]:
score = model2.evaluate(test_images2, test_labels2, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# You can now launch tensorboard with `tensorboard --logdir=./logs` on your
# command line and then go to http://localhost:6006/#projector to view the
# embeddings

Test loss: 0.07124984702132642
Test accuracy: 0.9782


In [12]:
import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)